In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-dev20200315


In [1]:
# Implementation of AlexNet in Functional API mode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, datasets, utils
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dropout

#tf.keras.backend.clear_session()
 
(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis=-1)
y_test = tf.squeeze(y_test, axis=-1)

def preprocess(x, y):
    x = 2 * tf.cast(x, dtype = tf.float32)/255. -1
    y = tf.cast(y, dtype = tf.int32)
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(128)
print("Data sets constructed!")
print(train_db, test_db)

alex_net_inputs = Input(shape=[32, 32, 3])
x = Conv2D(filters = 96, kernel_size=[11,11], strides = 4, padding = [[0,0],[2,2], [2,2],[0,0]], activation='relu')(alex_net_inputs)
x = MaxPool2D(pool_size=[3,3], strides = 2, padding='valid')(x)

x = Conv2D(filters = 256, kernel_size = [5,5], strides = 1, padding = [[0,0],[2,2], [2,2],[0,0]], activation = 'relu')(x)
x = MaxPool2D(pool_size=[3,3], strides = 2, padding='valid')(x)

x = Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)
x = Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)
x = Conv2D(filters = 256, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)

x = MaxPool2D(pool_size=[3,3], strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)

x = Dense(4096, activation='relu')(x)
x = Dropout(rate = 0.2)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate = 0.2)(x)
alex_net_outputs = Dense(10, activation='softmax')(x)



model = Model(inputs = alex_net_inputs, outputs = alex_net_outputs, name ='alexnet')
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #optimizer=keras.optimizers.RMSprop(),
              optimizer=keras.optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

history = model.fit(train_db, 
                    batch_size=64,
                    epochs=100)

test_scores = model.evaluate(test_db, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
#
#utils.plot_model(model_alexnet, 'test.png')


Data sets constructed!
<BatchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int32)> <BatchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int32)>
Epoch 1/100
391/391 [==============================] - 5s 13ms/step - loss: 2.1027 - accuracy: 0.3508
Epoch 2/100
391/391 [==============================] - 5s 13ms/step - loss: 2.0012 - accuracy: 0.4544
Epoch 3/100
391/391 [==============================] - 5s 13ms/step - loss: 1.9690 - accuracy: 0.4862
Epoch 4/100
391/391 [==============================] - 5s 13ms/step - loss: 1.9464 - accuracy: 0.5102
Epoch 5/100
391/391 [==============================] - 5s 12ms/step - loss: 1.9280 - accuracy: 0.5289
Epoch 6/100
391/391 [==============================] - 5s 13ms/step - loss: 1.9232 - accuracy: 0.5347
Epoch 7/100
391/391 [==============================] - 5s 13ms/step - loss: 1.9196 - accuracy: 0.5382
Epoch 8/100
391/391 [==============================] - 5s 13ms/step - loss: 1.9064 - accuracy: 0.

KeyboardInterrupt: 

In [1]:
# Implementation of VGG16 in Functional API mode
from numpy import random
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Flatten, Dropout
from tensorflow.keras import datasets, optimizers, losses, utils, Model, Input

(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis = -1)
y_test = tf.squeeze(y_test, axis = -1)

x = 2 * x.reshape(x.shape[0],32,32,3).astype('float32') / 255 - 1
#random.shuffle(x)
x_test = 2* x_test.reshape(x_test.shape[0],32,32,3).astype('float32')/ 255 -1

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.batch(128)

vgg16_inputs = Input(shape=[32,32,3])
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(vgg16_inputs)
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)

vgg16_outputs = Dense(10, activation=None)(x)

model_vgg16 = Model(vgg16_inputs, vgg16_outputs, name = 'VGG16')
model_vgg16.summary()

# CategoricalCrossentropy for one-hot encoded y, e.g. [0,1,0]
# SparseCategoricalCrossentropy for integer labels, e.g. [2, 3]
model_vgg16.compile(optimizer = optimizers.Adam(lr=1e-4), 
              loss = losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics = ['accuracy'])

training_log = model_vgg16.fit(train_db, batch_size = 128, epochs = 100)

testing_results = model_vgg16.evaluate(test_db, verbose = 2)
print('Test loss: ', testing_results[0])
print('Test Accuracy: ', testing_results[1])

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0     

KeyboardInterrupt: 

In [1]:
# Implementation of VGG16 in Functional API mode with Caltech101 data

# Implementation of VGG16 in Functional API mode
from numpy import random
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Flatten, Dropout
from tensorflow.keras import datasets, optimizers, losses, utils, Model, Input
import tensorflow_datasets as tfds

dataset_name = 'caltech101'
ds_info = tfds.builder(dataset_name).info
num_classes = ds_info.features['label'].num_classes
batch_size = 32 
epochs = 50 
#ds_train, ds_test = tfds.load('caltech101', split = ['train[:20%]', 'test[:10%]'])
#ds_train, ds_test = tfds.load('caltech101', split = ['test[:90%]','test[-10%:]'])
#ds_train, ds_test = tfds.load(dataset_name, split = ['train', 'test'])
ds_train, ds_test = tfds.load(dataset_name, split = ['test','train'])

def preprocess(x,y):
    x = 2*tf.cast(x, dtype=tf.float32)/255. - 1
    y = tf.cast(y,dtype = tf.int32)
    return x,y

def getDb(ds):
    x, y = [],[]
    for sample in ds:
        x.append(tf.image.resize(sample["image"], size = [224,224]))
        y.append(sample["label"])
    db = tf.data.Dataset.from_tensor_slices((x, y))
    print(len(x))
    return db

train_db = getDb(ds_train)
train_db = train_db.shuffle(100).map(preprocess).batch(batch_size)
test_db = getDb(ds_test)
test_db = test_db.map(preprocess).batch(batch_size)
print("Data set", dataset_name, " has been constructed!")
print("Num of Classes:",num_classes)
#print(train_db, test_db)

vgg16_inputs = Input(shape=[224,224,3])
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(vgg16_inputs)
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)

vgg16_outputs = Dense(num_classes, activation=None)(x)

model_vgg16 = Model(vgg16_inputs, vgg16_outputs, name = 'VGG16')
# model_vgg16.summary()

# CategoricalCrossentropy for one-hot encoded y, e.g. [0,1,0]
# SparseCategoricalCrossentropy for integer labels, e.g. [2, 3]
model_vgg16.compile(optimizer = optimizers.Adam(lr=1e-4),
               loss = losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])
# 
training_log = model_vgg16.fit(train_db, batch_size = batch_size, epochs = epochs)


testing_results = model_vgg16.evaluate(test_db, verbose = 2)
print('Test loss: ', testing_results[0])
print('Test Accuracy: ', testing_results[1])

6084
3060
Data set caltech101  has been constructed!
Num of Classes: 102
Epoch 1/50
191/191 [==============================] - 40s 210ms/step - loss: 3.6051 - accuracy: 0.1872
Epoch 2/50
191/191 [==============================] - 39s 207ms/step - loss: 2.7979 - accuracy: 0.3979
Epoch 3/50
191/191 [==============================] - 40s 208ms/step - loss: 2.4185 - accuracy: 0.4860
Epoch 4/50
191/191 [==============================] - 40s 209ms/step - loss: 2.1251 - accuracy: 0.5345
Epoch 5/50
191/191 [==============================] - 40s 210ms/step - loss: 1.7569 - accuracy: 0.5934
Epoch 6/50
191/191 [==============================] - 40s 209ms/step - loss: 1.4452 - accuracy: 0.6524
Epoch 7/50
191/191 [==============================] - 40s 210ms/step - loss: 1.1308 - accuracy: 0.7222
Epoch 8/50
191/191 [==============================] - 40s 210ms/step - loss: 0.7981 - accuracy: 0.7908
Epoch 9/50
191/191 [==============================] - 40s 210ms/step - loss: 0.5666 - accuracy: 0.8486


In [1]:

import torch
print(torch.__version__)

1.5.0


In [ ]:
def main():
    generator = Generator()
    generator.build(input_shape=(None, z_dim))
    #generator.summary()
    
    discriminator = Discriminator()
    discriminator.build(input_shape=(None, 64, 64, 3))
    #discriminator.summary()
    
    g_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    d_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
    
    d_losses, g_losses = [], []
    
    for epoch in range(epochs):
        for _ in range(1):
            batch_z = get_random_z(z_dim, batch_size)
            batch_x = next(db_iter)
            
            with tf.GradientTape() as d_tape:
                d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
            d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        
        batch_zz = get_random_z(z_dim, batch_size)
        batch_xx = next(db_iter)
        
        with tf.GradientTape() as g_tape:
            g_loss = g_loss_fn(generator, discriminator, batch_zz, is_training)
        g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))
            
        if epoch % 100 == 0:
            print("Epoch:", epoch, "D-loss:", float(d_loss), "G-loss:", float(g_loss))
            z = tf.random.uniform([100, z_dim], minval=-1, maxval=1)
            generated_images = generator(z, training = False)
            img_path = os.path.join('gan_images', 'gan-%d.png'%epoch)
            save_result(generated_images.numpy(), 10, img_path, color_mode='P')

            d_losses.append(float(d_loss))
            g_losses.append(float(g_loss))

        if epoch % 10000 == 1:
            generator.save_weights('generator.ckpt')
            discriminator.save_weights('discriminator.ckpt')
if __name__ == "__main__":
    main()